In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, Normalizer, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore') # 경고 메시지 무시

# titanic 데이터셋은 기존 대회에서 주어진 데이터셋으로 train, test가 분할되어 배포됨. 
# 두 개의 데이터셋을 불러와 concat()을 통해 합쳐야함.
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# 데이터셋 합치기
df = pd.concat([df_train, df_test], axis = 0)

# 데이터셋 기본 정보 확인
print(df.head())
df.info()

# 결측치 확인
print(df.isnull().sum())

# 데이터 복사본 생성해서 사용할 경우
# df_copy = df.copy(deep=True)

   PassengerId  Survived  Pclass  \
0            1       0.0       3   
1            2       1.0       1   
2            3       1.0       3   
3            4       1.0       1   
4            5       0.0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
<c

In [2]:
# Age 결측치 처리: 중앙값으로 대체
imputer_age = SimpleImputer(strategy='median')
df['Age'] = imputer_age.fit_transform(df[['Age']])
print("\nAge 결측치 처리 후")
print(f"Age 컬럼 결측치 개수: {df['Age'].isnull().sum()}")

# Embarked 결측치 처리: 최빈값으로 대체
imputer_embarked = SimpleImputer(strategy='most_frequent')
# .squeeze()를 사용하여 1차원 배열로 변환
df['Embarked'] = imputer_embarked.fit_transform(df[['Embarked']]).squeeze()
print("\nEmbarked 결측치 처리 후")
print(f"Embarked 컬럼 결측치 개수: {df['Embarked'].isnull().sum()}")

# Cabin 결측치 처리: 'N'으로 대체
df['Cabin'] = df['Cabin'].fillna('N')
print("\nCabin 결측치 처리 후")
print(f"Cabin 컬럼 결측치 개수: {df['Cabin'].isnull().sum()}")

print("\n모든 결측치 처리 후 최종 확인")
print(df.isnull().sum())


Age 결측치 처리 후
Age 컬럼 결측치 개수: 0

Embarked 결측치 처리 후
Embarked 컬럼 결측치 개수: 0

Cabin 결측치 처리 후
Cabin 컬럼 결측치 개수: 0

모든 결측치 처리 후 최종 확인
PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin            0
Embarked         0
dtype: int64


In [3]:
# KNN 대체
#	결측치가 전체 데이터 중 5~10% 이내일 때 성능이 좋음
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_numeric = df.select_dtypes(include=[np.number])
df_numeric_imputed = pd.DataFrame(imputer.fit_transform(df_numeric), columns=df_numeric.columns)

# MICE 대체 (Multiple Imputation by Chained Equations)
# 각 변수를 다른 변수로 예측하는 방식이므로, 변수 간 상관성이 높을수록 효과적
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
mice_imputer = IterativeImputer(random_state=42)
df_numeric_mice = pd.DataFrame(mice_imputer.fit_transform(df_numeric), columns=df_numeric.columns)

In [4]:
# Sex 컬럼 레이블 인코딩
# 'male'을 0, 'female'을 1 등으로 변환
le = LabelEncoder() #인코더 생성
df['Sex_encoded'] = le.fit_transform(df['Sex'])
print("\nSex 레이블 인코딩 예시")
print(df[['Sex', 'Sex_encoded']].head())
print(f"인코딩된 값: {le.classes_} -> {le.transform(le.classes_)}") # 어떤 값이 어떻게 변환되었는지 확인

# Embarked 컬럼 원-핫 인코딩
# 원-핫 인코딩은 새로운 컬럼을 생성하므로, 기존 데이터프레임에 병합합니다.
# drop_first=True로 설정하면 다중 공선성 문제를 피할 수 있지만, 여기서는 이해를 위해 False로 둡니다.
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# 희소 행렬(sparse matrix)을 반환하지만, 이걸 일반적인 NumPy 배열로 바꾸기 위해 사용
# 학습되지 않은 새로운 범주가 예측에 들어올 경우 오류를 내는 대신 0벡터로 처리함
embarked_encoded = ohe.fit_transform(df[['Embarked']])

# OneHotEncoder의 출력은 NumPy 배열이므로 DataFrame으로 변환
# get_feature_names_out()을 사용하여 컬럼 이름 생성
embarked_df = pd.DataFrame(embarked_encoded, columns=ohe.get_feature_names_out(['Embarked']))

# 원본 데이터프레임과 병합
df = pd.concat([df.reset_index(drop=True), embarked_df], axis=1)
# 열 방향으로 병합

print("\nEmbarked 원-핫 인코딩 예시")
print(df[['Embarked', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].head())


Sex 레이블 인코딩 예시
      Sex  Sex_encoded
0    male            1
1  female            0
2  female            0
3  female            0
4    male            1
인코딩된 값: ['female' 'male'] -> [0 1]

Embarked 원-핫 인코딩 예시
  Embarked  Embarked_C  Embarked_Q  Embarked_S
0        S         0.0         0.0         1.0
1        C         1.0         0.0         0.0
2        S         0.0         0.0         1.0
3        S         0.0         0.0         1.0
4        S         0.0         0.0         1.0


In [5]:
# FamilySize 파생 변수 생성
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
print("\nFamilySize 파생 변수 예시")
print(df[['SibSp', 'Parch', 'FamilySize']].head())

# IsAlone 파생 변수 생성
# FamilySize가 1이면 1 (혼자), 아니면 0 (혼자가 아님)
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
print("\nIsAlone 파생 변수 예시")
print(df[['FamilySize', 'IsAlone']].head())

# Title 파생 변수 생성 (이름에서 호칭 추출)
# 정규표현식을 사용하여 이름에서 호칭 추출
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# 드물게 나타나는 호칭들을 'Rare'로 묶기
rare_titles = ['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
df['Title'] = df['Title'].replace(rare_titles, 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss') # Mlle는 Miss로
df['Title'] = df['Title'].replace('Ms', 'Miss')   # Ms도 Miss로
df['Title'] = df['Title'].replace('Mme', 'Mrs')   # Mme는 Mrs로

print("\nTitle 파생 변수 예시 및 분포")
print(df['Title'].value_counts())

# Title도 범주형이므로 레이블 인코딩
df['Title_encoded'] = le.fit_transform(df['Title']) # 기존 LabelEncoder 재활용
print("\nTitle 레이블 인코딩 예시")
print(df[['Title', 'Title_encoded']].head())


FamilySize 파생 변수 예시
   SibSp  Parch  FamilySize
0      1      0           2
1      1      0           2
2      0      0           1
3      1      0           2
4      0      0           1

IsAlone 파생 변수 예시
   FamilySize  IsAlone
0           2        0
1           2        0
2           1        1
3           2        0
4           1        1

Title 파생 변수 예시 및 분포
Title
Mr        757
Miss      264
Mrs       198
Master     61
Rare       29
Name: count, dtype: int64

Title 레이블 인코딩 예시
  Title  Title_encoded
0    Mr              2
1   Mrs              3
2  Miss              1
3   Mrs              3
4    Mr              2


In [8]:
# family_size를 가지고 family group이라는 새로운 범주형 범수 생성 가능
def map_family_group(size):
    if size == 1:
        return 'Solo'
    elif 2 <= size <= 4:
        return 'Small'
    else:
        return 'Large'
        
df['FamilyGroup'] = df['FamilySize'].apply(map_family_group)

In [7]:
# 스케일링할 숫자형 특성들 정의
numeric_features_to_scale = ['Age', 'Fare']

# 1. StandardScaler (표준화) 적용
scaler_standard = StandardScaler()
# fit_transform은 2D 배열을 기대하므로, 데이터프레임 슬라이싱 형태 [[]] 유지
scaled_data_standard = scaler_standard.fit_transform(df[numeric_features_to_scale])
#fit은 데이터의 평균과 표준편차를 학습하고, transform은 학습된 값으로 데이터를 변환
df_standard_scaled = pd.DataFrame(scaled_data_standard, columns=numeric_features_to_scale)
df_standard_scaled = df_standard_scaled.add_suffix('_standard_scaled') # 모든 컬럼 이름에 접미사 추가
#변환된 데이터를 새로운 데이터프레임으로

# 원본 데이터프레임에 병합
df = pd.concat([df.reset_index(drop=True), df_standard_scaled], axis=1)

print("\nStandardScaler 적용 후")
print(df[[f'{col}_standard_scaled' for col in numeric_features_to_scale]].head())


# 2. MinMaxScaler (정규화) 적용
min_max_scaler = MinMaxScaler()
scaled_data_minmax = min_max_scaler.fit_transform(df[numeric_features_to_scale]) # 변환된 NumPy 배열 / fit_transform()은 2차원 배열 기대 -> df 

df_minmax_scaled = pd.DataFrame(scaled_data_minmax, columns=numeric_features_to_scale)
# 변환된 데이터를 데이터프레임으로 변환
# 새로 생성될 데이터프레임 칼럼 이름 지정
df_minmax_scaled = df_minmax_scaled.add_suffix('_minmax_scaled')
# 모든 칼럼 뒤에 접미사 추가



# 원본 데이터프레임에 병합
df = pd.concat([df.reset_index(drop=True), df_minmax_scaled], axis=1)
#기존 인덱스 버리고, 새로운 인덱스 추가

print("\nMinMaxScaler 적용 후")
print(df[[f'{col}_minmax_scaled' for col in numeric_features_to_scale]].head())

# RobustScaler, MaxAbsScaler, Normalizer도 유사한 방식으로 적용가능



StandardScaler 적용 후
   Age_standard_scaled  Fare_standard_scaled
0            -0.581628             -0.503402
1             0.658652              0.734222
2            -0.271558             -0.490356
3             0.426099              0.382778
4             0.426099             -0.487940

MinMaxScaler 적용 후
   Age_minmax_scaled  Fare_minmax_scaled
0           0.273456            0.014151
1           0.473882            0.139136
2           0.323563            0.015469
3           0.436302            0.103644
4           0.436302            0.015713
